In [ ]:
import obspy
from obspy import read
from obspy import Trace, Stream
import h5py
from obspy import UTCDateTime
import numpy as np
from obspy.clients.fdsn.client import Client
import matplotlib.pyplot as plt

from matplotlib import mlab
from matplotlib.colors import Normalize
from obspy.imaging.cm import obspy_sequential
import os

import string
import argparse as ap
import sys
import os
import numpy as np

import obspy
import obspy.core as oc
from obspy import read_inventory

# from datetime import datetime, timedelta
import time

import pylab as plt
import pandas as pd
import h5py


import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import datetime
import warnings

mpl.rcParams['pdf.fonttype'] = 42



In [ ]:
from google.colab import drive  # Import for Google Drive access
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

def build_stream(df, h5, filt, freq_min, freq_max):

    all_st = obspy.Stream()  # Create an empty stream

    for line in range(len(df)):  # Loop over all events

        row = df.iloc[line, :]
        waveform = h5['data'][row['trace_name']]
        stats = oc.Stats()
        stats.npts = 12000
        stats.sampling_rate = 100.

        sta = row['station_code']
        net = row['station_network_code']



        stats.delta = row['trace_dt_s']
        stats.starttime = UTCDateTime(row['trace_start_time'])  # Use ObsPy UTCDateTime
        stats.network = net
        stats.station = sta

        stats.category = row['trace_category']

        if row['trace_P_arrival_time'] in ["", "None", "null"]:
            stats.p_arrival_time = None
        else:
            stats.p_arrival_time = UTCDateTime(row['trace_P_arrival_time'])

        if row['trace_S_arrival_time'] in ["", "None", "null"]:
            stats.s_arrival_time = None
        else:
            stats.s_arrival_time = UTCDateTime(row['trace_S_arrival_time'])


        st = obspy.Stream()
        for i in range(3):
            tr = obspy.Trace()
            tr.data = waveform[i]
            tr.stats = stats.copy()  # Copy stats to avoid overwriting

            if i == 0:
                tr.stats.channel = row['station_channels'] + 'E'
            elif i == 1:
                tr.stats.channel = row['station_channels'] + 'N'
            elif i == 2:
                tr.stats.channel = row['station_channels'] + 'Z'

            st.append(tr)

        all_st += st  # Append the event stream to the main stream

    if filt:
        all_st.detrend(type='linear')
        all_st.filter(type='bandpass', freqmin=freq_min, freqmax=freq_max)

    return all_st, row

def counts_to_vel(df, h5, filt, freq_min, freq_max, client):

    all_st = obspy.Stream()  # Create an empty stream
    failed_events = []  # List to track failed response removals

    for line in range(len(df)):  # Loop over all events
        row = df.iloc[line, :]

        stats = oc.Stats()
        stats.npts = 12000
        stats.sampling_rate = 100.0

        sta = row['station_code']
        net = row['station_network_code']
        trace_name = row['trace_name']
        waveform = h5['data'][trace_name]

        stats.delta = row['trace_dt_s']
        stats.starttime = UTCDateTime(row['trace_start_time'])  # Convert to ObsPy time
        stats.network = net
        stats.station = sta
        stats.category = row['trace_category']
        stats.p_arrival_time = None
        stats.s_arrival_time = None

        # Fetch response metadata
        try:
            inventory = client.get_stations(
                network=net,
                station=sta,
                starttime=stats.starttime,
                endtime=stats.starttime + 120,  # Add 120 seconds
                loc="*",
                channel="*",
                level="response"
            )
        except Exception as e:
            print(f"Failed to get response for event {trace_name}: {e}")
            failed_events.append(trace_name)
            continue  # Skip this event if inventory retrieval fails

        st = obspy.Stream()
        for i in range(3):
            tr = obspy.Trace()
            tr.data = waveform[i]
            tr.stats = stats.copy()  # Copy stats to avoid modifying reference

            if i == 0:
                tr.stats.channel = row['station_channels'] + 'E'
            elif i == 1:
                tr.stats.channel = row['station_channels'] + 'N'
            elif i == 2:
                tr.stats.channel = row['station_channels'] + 'Z'

            st.append(tr)

        # Try to remove response
        try:
            st.remove_response(inventory=inventory, output="VEL", plot=False)
        except Exception as e:
            failed_events.append(trace_name)
            continue  # Skip this event if response removal fails

        all_st += st  # Append processed event stream to the main stream

    # Apply filtering if requested
    if filt:
        all_st.detrend(type='linear')
        all_st.filter(type='bandpass', freqmin=freq_min, freqmax=freq_max)

    return all_st, failed_events, row


def split(word):
    chars = [char for char in word]
    charsout = []
    for c in chars:
        charsout.append('(' + c + ')')
    return charsout



In [ ]:

def trace_slice(original_trace):
    p_arrival_time = original_trace.stats.p_arrival_time
    s_arrival_time = original_trace.stats.s_arrival_time

    # Split into 4 slices of 3000 samples (30 seconds each at 100Hz)
    samples_per_window = 3000
    sampling_rate = original_trace.stats.sampling_rate

    sliced_traces = Stream()

    for i in range(4):
        start_sample = i * samples_per_window
        end_sample = start_sample + samples_per_window

        sliced_data = original_trace.data[start_sample:end_sample]

        new_start_time = original_trace.stats.starttime + (start_sample / sampling_rate)

        new_trace = Trace(data=sliced_data, header=original_trace.stats.copy())
        new_trace.stats.starttime = new_start_time
        new_trace.stats.npts = len(sliced_data)

        # P-phase
        if new_start_time <= p_arrival_time <= new_trace.stats.endtime:
            relative_p_arrival = p_arrival_time - new_start_time
            print(f"Slice {i}: P-phase at {relative_p_arrival:.2f} seconds in window")
            new_trace.stats.p_phase_relative_time = relative_p_arrival
        else:
            new_trace.stats.p_phase_relative_time = None

        # S-phase
        if new_start_time <= s_arrival_time <= new_trace.stats.endtime:
            relative_s_arrival = s_arrival_time - new_start_time
            print(f"Slice {i}: S-phase at {relative_s_arrival:.2f} seconds in window")
            new_trace.stats.s_phase_relative_time = relative_s_arrival
        else:
            new_trace.stats.s_phase_relative_time = None

        sliced_traces.append(new_trace)

    return sliced_traces

def stream_slice(stream):
    sliced_stream = Stream()
    for trace in stream:
        sliced_traces = trace_slice(trace)
        sliced_stream.extend(sliced_traces)
    return sliced_stream


In [ ]:

def trace_slice_noise(original_trace):
    #p_arrival_time = original_trace.stats.p_arrival_time
    #s_arrival_time = original_trace.stats.s_arrival_time

    # Split into 4 slices of 3000 samples (30 seconds each at 100Hz)
    samples_per_window = 3000
    sampling_rate = original_trace.stats.sampling_rate

    sliced_traces = Stream()

    for i in range(4):
        start_sample = i * samples_per_window
        end_sample = start_sample + samples_per_window

        sliced_data = original_trace.data[start_sample:end_sample]

        new_start_time = original_trace.stats.starttime + (start_sample / sampling_rate)

        new_trace = Trace(data=sliced_data, header=original_trace.stats.copy())
        new_trace.stats.starttime = new_start_time
        new_trace.stats.npts = len(sliced_data)
        sliced_traces.append(new_trace)

        """
        # P-phase
        if new_start_time <= p_arrival_time <= new_trace.stats.endtime:
            relative_p_arrival = p_arrival_time - new_start_time
            print(f"Slice {i}: P-phase at {relative_p_arrival:.2f} seconds in window")
            new_trace.stats.p_phase_relative_time = relative_p_arrival
        else:
            new_trace.stats.p_phase_relative_time = None

        # S-phase
        if new_start_time <= s_arrival_time <= new_trace.stats.endtime:
            relative_s_arrival = s_arrival_time - new_start_time
            print(f"Slice {i}: S-phase at {relative_s_arrival:.2f} seconds in window")
            new_trace.stats.s_phase_relative_time = relative_s_arrival
        else:
            new_trace.stats.s_phase_relative_time = None

        sliced_traces.append(new_trace)
        """

    return sliced_traces

def stream_slice_noise(stream):
    sliced_stream = Stream()
    for trace in stream:
        sliced_traces = trace_slice_noise(trace)
        sliced_stream.extend(sliced_traces)
    return sliced_stream


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.mlab import specgram

def compute_dominant_frequencies(data, fs=100.0, window_length=200, hop_size=14, desired_frames=200, plot=False, title=None):
    """
    Compute STFT and extract dominant frequency per timeframe.

    Parameters:
        data (np.ndarray): Input 1D signal.
        fs (float): Sampling frequency in Hz.
        window_length (int): STFT window size.
        desired_frames (int): Number of desired timeframes.
        plot (bool): If True, show the spectrogram and dominant frequencies.

    Returns:
        time_bins (np.ndarray): Time (in seconds) for each STFT frame.
        dominant_freqs (np.ndarray): Dominant frequency for each timeframe.
    """
    #signal = data
    signal = data - np.mean(data)

    # Calculate hop size to get desired number of frames
    hop_size = hop_size
    noverlap = window_length - hop_size
    window = np.hanning(window_length)

    Pxx, freqs, time_bins = specgram(
        signal,
        NFFT=window_length,
        Fs=fs,
        window=window,
        noverlap=noverlap
    )

    #Pxx = Pxx[1:, :]     # remove DC component
    #freqs = freqs[1:]    # remove DC frequency

    dominant_indices = np.argmax(Pxx, axis=0)
    dominant_freqs = freqs[dominant_indices]

    # Ensure exactly desired_frames by slicing the first 200
    time_bins = time_bins[:desired_frames]
    dominant_freqs = dominant_freqs[:desired_frames]
    Pxx = Pxx[:, :desired_frames]  # Slice the spectrogram data too

    #time_bins = time_bins[:]
    #dominant_freqs = dominant_freqs[:]
    #Pxx = Pxx[:, :]  # Slice the spectrogram data too

    if plot:
        plt.figure(figsize=(10, 4))
        plt.pcolormesh(time_bins, freqs, Pxx, shading='gouraud')
        plt.plot(time_bins, dominant_freqs, color='red', label='Dominant Frequency')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [s]')
        plt.title(f"{title} - Spectrogram (Linear Power)")
        plt.colorbar(label='Power')
        plt.legend()
        plt.tight_layout()
        plt.show()

    return time_bins, dominant_freqs


In [ ]:

def trace_dominant_frequencies(trace, **kwargs):
    """
    Extract dominant frequencies from a seismic trace.

    Parameters:
    - trace (obspy.Trace): Seismic trace from which to extract dominant frequencies.
    - kwargs: Additional parameters for `extract_dominant_frequencies`.

    Returns:
    - List of dominant frequencies extracted every 0.2 seconds.
    """
    freq = []
    freq_modified = []
    time_modified = []

    if 'title' not in kwargs:
            kwargs['title'] = str(trace.id)

    freq = compute_dominant_frequencies(data=trace.data, **kwargs)

    for i in range(len(freq)):
      if i == 0:
        for idx in range(len(freq[i])):
          for j in range(15):
            time_modified.append(freq[i][idx])

      if i == 1:
        for idx in range(len(freq[i])):
          for j in range(15):
            freq_modified.append(freq[i][idx])

    index = np.arange(len(time_modified)) * 0.01

    return index, freq_modified #seharusnya return index t


def stream_dominant_frequencies(stream, **kwargs):
    """
    Extract dominant frequencies for each trace in the given stream.

    Parameters:
    - stream (obspy.Stream): The seismic stream containing multiple traces.
    - kwargs: Additional parameters for `trace_dominant_frequencies`.

    Returns:
    - List of dominant frequency lists, one for each trace in the stream.

    Example:
    >>> from obspy import read
    >>> st = read()
    >>> freqs = stream_dominant_frequencies(st)
    """
    spec_list = []
    for tr in stream:
        spec = trace_dominant_frequencies(tr, **kwargs)
        spec_list.append(spec)

    return spec_list

In [ ]:
def create_dataset(data):
    """
    Extract dominant frequencies for each trace in the given stream.

    Parameters:
    - stream (obspy.Stream): The seismic stream containing multiple traces.
    - kwargs: Additional parameters for `trace_dominant_frequencies`.

    Returns:
    - List of dominant frequency lists, one for each trace in the stream.

    Example:
    >>> from obspy import read
    >>> st = read()
    >>> freqs = stream_dominant_frequencies(st)
    """
    # Convert to LSTM-friendly format
    X = np.array([features for _, features in data])

    # Reshape to (num_samples, timesteps, num_features)
    X = X.reshape(X.shape[0], X.shape[1], 1)  # Assuming 1 feature per timestep

    print(X.shape)  # Expected: (num_samples, timesteps, num_features)

    return X

def trace_labels(trace, time, show=False):
    import numpy as np
    import matplotlib.pyplot as plt

    duration = trace.stats.endtime - trace.stats.starttime

    t_start = 0
    t_end = t_start + duration
    #sampling_rate = trace.stats.sampling_rate

    num_samples = len(trace.data)
    t = np.linspace(t_start, t_end, num_samples, endpoint=True)
    #t = np.array(time)  # ✅ Ensure time is a NumPy array
    square_wave = np.zeros_like(t)

    p_arrival = trace.stats.p_phase_relative_time
    s_arrival = trace.stats.s_phase_relative_time

    if p_arrival is not None:
        pulse_start = t_start + p_arrival

        if s_arrival is not None:
            duration_after = s_arrival - p_arrival
            pulse_end = pulse_start + duration_after
        else:
            pulse_end = t_end

        square_wave[(t >= pulse_start) & (t <= pulse_end)] = 1
    else:
        pulse_start = 0
        pulse_end = 0

    if show:
        plt.plot(np.arange(len(square_wave)), square_wave, label="Ground Truth")
        plt.xlabel("Sample Index")
        plt.ylabel("Amplitude")
        plt.title("Square Impulse Wave")
        plt.legend()
        plt.grid(True)
        plt.show()

    return t, square_wave

def stream_labels(stream, time, show=False):
    """
    Extract dominant frequencies for each trace in the given stream.

    Parameters:
    - stream (obspy.Stream): The seismic stream containing multiple traces.
    - kwargs: Additional parameters for `trace_dominant_frequencies`.

    Returns:
    - List of dominant frequency lists, one for each trace in the stream.

    Example:
    >>> from obspy import read
    >>> st = read()
    >>> freqs = stream_dominant_frequencies(st)
    """
    labels_list = []
    for tr in stream:
        labels = trace_labels(tr, time, show)
        labels_list.append(labels)

    return labels_list

def noise_labels(trace, time, show=False):
    duration = trace.stats.endtime - trace.stats.starttime

    t_start = 0
    t_end = t_start + duration
    #sampling_rate = trace.stats.sampling_rate

    # Convert sample count to an integer
    num_samples = len(trace.data)

    t = np.linspace(t_start, t_end, num_samples, endpoint=True)
    #t = time
    square_wave = np.zeros_like(t)
    """
    if trace.stats.p_arrival_time is not None:
      duration_before = trace.stats.p_arrival_time - trace.stats.starttime
      pulse_start = t_start + duration_before

      if trace.stats.s_arrival_time is not None:
        duration_after = trace.stats.s_arrival_time - trace.stats.p_arrival_time
        pulse_end = pulse_start + duration_after
      else:
        pulse_end = t_end

      square_wave[(t >= pulse_start) & (t <= pulse_end)] = 0

    else:
      pulse_start = 0
      pulse_end = 0
    """
    pulse_start = 0
    pulse_end = 0
    square_wave[(t >= pulse_start) & (t <= pulse_end)] = 0

    if show:
        plt.plot(np.arange(len(square_wave)), square_wave, label="Ground Truth")
        plt.xlabel("Sample Index")
        plt.ylabel("Amplitude")
        plt.title("Square Impulse Wave")
        plt.legend()
        plt.grid(True)
        plt.show()

    return t, square_wave

def noise_stream_labels(stream, time, show=False):
    """
    Extract dominant frequencies for each trace in the given stream.

    Parameters:
    - stream (obspy.Stream): The seismic stream containing multiple traces.
    - kwargs: Additional parameters for `trace_dominant_frequencies`.

    Returns:
    - List of dominant frequency lists, one for each trace in the stream.

    Example:
    >>> from obspy import read
    >>> st = read()
    >>> freqs = stream_dominant_frequencies(st)
    """
    labels_list = []
    for tr in stream:
        labels = noise_labels(tr, time, show)
        labels_list.append(labels)

    return labels_list




In [ ]:
warnings.filterwarnings("ignore")
mpl.rcParams['pdf.fonttype'] = 42

FIGDIR='/content/drive/MyDrive/data_codingan_skripsi/figure'


# file NAMES
# MetaData
events_metaname=os.path.join('/content/drive/MyDrive/data_codingan_skripsi/events_filtered_metadata.csv')
#noise_metaname=os.path.join('/content/drive/MyDrive/data_codingan_skripsi/noise_metadata.csv')
# HDF5
events_hdfname=os.path.join('/content/drive/MyDrive/data_codingan_skripsi/merged_events.h5')
#noise_hdfname=os.path.join('/content/drive/MyDrive/data_codingan_skripsi/merged_noise_data.h5')
# Figures
# events_figname=os.path.join(FIGDIR,'waveforms_Evs.png')
# gm_figname=os.path.join(FIGDIR,'waveforms_Evs_gm.png')
# noise_figname=os.path.join(FIGDIR,'waveforms_Noise.png')

events_metaData = pd.read_csv(events_metaname, keep_default_na=False, dtype={'station_location_code': object,
           'source_mt_eval_mode':object,'source_mt_status': object,'source_mechanism_strike_dip_rake': object,
           'source_mechanism_moment_tensor': object, 'trace_p_arrival_time': object, 'trace_s_arrival_time': object},
                             low_memory=False)

#noise_metaData = pd.read_csv(noise_metaname,dtype={'station_location_code': object}, low_memory=False)

events_h5File = h5py.File(events_hdfname, 'r') # Events in gm
#noise_h5File = h5py.File(noise_hdfname, 'r')   # Noise in counts

In [ ]:
events_metaData.drop_duplicates(subset='trace_name', keep='first', inplace=True)
events_metaData.reset_index(drop=True, inplace=True)

In [ ]:
events, row_events = build_stream(events_metaData,events_h5File, False, 0, 10000)


In [ ]:
noise = read("/content/drive/MyDrive/data_codingan_skripsi/noise_gm.mseed", format="MSEED")

In [ ]:
noise.filter("bandpass", freqmin=1.0, freqmax=45.0)


2607 Trace(s) in Stream:

IV.ORZI..HNE | 2013-12-05T21:08:54.345000Z - 2013-12-05T21:10:54.335000Z | 100.0 Hz, 12000 samples
...
(2605 other traces)
...
IV.EPZF..HHZ | 2013-12-22T03:44:55.410000Z - 2013-12-22T03:46:55.400000Z | 100.0 Hz, 12000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [ ]:
events.filter("bandpass", freqmin=1.0, freqmax=45.0)


16230 Trace(s) in Stream:

IV.OFFI..HHE | 2016-12-04T15:34:43.920000Z - 2016-12-04T15:36:43.910000Z | 100.0 Hz, 12000 samples
...
(16228 other traces)
...
IV.VCEL..EHZ | 2016-12-09T12:12:53.260000Z - 2016-12-09T12:14:53.250000Z | 100.0 Hz, 12000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [ ]:
len(events)

16230

In [ ]:
len(noise)

2607

In [ ]:
events_sliced = stream_slice(events)

Streaming output truncated to the last 5000 lines.
Slice 0: P-phase at 17.98 seconds in window
Slice 0: P-phase at 17.98 seconds in window
Slice 0: P-phase at 15.66 seconds in window
Slice 0: P-phase at 15.66 seconds in window
Slice 0: P-phase at 15.66 seconds in window
Slice 0: P-phase at 19.18 seconds in window
Slice 0: S-phase at 22.50 seconds in window
Slice 0: P-phase at 19.18 seconds in window
Slice 0: S-phase at 22.50 seconds in window
Slice 0: P-phase at 19.18 seconds in window
Slice 0: S-phase at 22.50 seconds in window
Slice 0: P-phase at 18.16 seconds in window
Slice 0: P-phase at 18.16 seconds in window
Slice 0: P-phase at 18.16 seconds in window
Slice 0: P-phase at 18.27 seconds in window
Slice 0: P-phase at 18.27 seconds in window
Slice 0: P-phase at 18.27 seconds in window
Slice 0: P-phase at 20.38 seconds in window
Slice 0: S-phase at 23.94 seconds in window
Slice 0: P-phase at 20.38 seconds in window
Slice 0: S-phase at 23.94 seconds in window
Slice 0: P-phase at 20.38

In [ ]:
noise_sliced = stream_slice_noise(noise)

In [ ]:
len(events_sliced)

64920

In [ ]:
len(noise_sliced)

10428

In [ ]:
del events

In [ ]:
del noise

# generate labels

In [ ]:
"""
t, freq = trace_dominant_frequencies(events_sliced[0], plot=False)
print(t[300])

"""

'\nt, freq = trace_dominant_frequencies(events_sliced[0], plot=False)\nprint(t[300])\n\n'

In [ ]:
"""
t1, freq = trace_dominant_frequencies(events_sliced[10], plot=False)
print(t1[300])
"""

'\nt1, freq = trace_dominant_frequencies(events_sliced[10], plot=False)\nprint(t1[300])\n'

In [ ]:
"""
Y_events = stream_labels(events_sliced, time=t, show=False)
"""

'\nY_events = stream_labels(events_sliced, time=t, show=False)\n'

In [ ]:
"""
len(Y_events)
"""

'\nlen(Y_events)\n'

In [ ]:
"""
Y_1 = create_dataset(Y_events)
"""

'\nY_1 = create_dataset(Y_events)\n'

In [ ]:
"""
np.save("Y_1_sliced.npy", Y_1)  # Save
"""

'\nnp.save("Y_1_sliced.npy", Y_1)  # Save\n'

In [ ]:
"""
import shutil

shutil.move("/content/Y_1_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi")
"""

'\nimport shutil\n\nshutil.move("/content/Y_1_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi")\n'

In [ ]:
"""
del Y_1
del Y_events
"""

'\ndel Y_1\ndel Y_events\n'

In [ ]:
"""
Y_noise = noise_stream_labels(noise_sliced, time=t, show=False)
"""

'\nY_noise = noise_stream_labels(noise_sliced, time=t, show=False)\n'

In [ ]:
"""
Y_2 = create_dataset(Y_noise)
"""

'\nY_2 = create_dataset(Y_noise)\n'

In [ ]:
"""
np.save("Y_2_sliced.npy", Y_2)  # Save
"""

'\nnp.save("Y_2_sliced.npy", Y_2)  # Save\n'

In [ ]:
"""
shutil.move("/content/Y_2_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi")
"""

'\nshutil.move("/content/Y_2_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi")\n'

In [ ]:
"""
del Y_2
del Y_noise
"""

'\ndel Y_2\ndel Y_noise\n'

# feature extraction

In [ ]:
sliced_combined = events_sliced + noise_sliced

In [ ]:
len(sliced_combined)

75348

In [ ]:
t, freq = trace_dominant_frequencies(events_sliced[0], plot=False)

In [ ]:
freq_1 = stream_dominant_frequencies(sliced_combined[:15000], plot=False)
X_1 = create_dataset(freq_1)
np.save("X1_STFT_sliced.npy", X_1)  # Save

(15000, 3000, 1)


In [ ]:
del freq_1
del X_1

In [ ]:
freq_1 = stream_dominant_frequencies(sliced_combined[15000:30000], plot=False)
X_1 = create_dataset(freq_1)
np.save("X2_STFT_sliced.npy", X_1)  # Save

(15000, 3000, 1)


In [ ]:
del freq_1
del X_1

In [ ]:
import shutil

# Move file to Google Drive
shutil.move("/content/X1_STFT_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames")
shutil.move("/content/X2_STFT_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames")

'/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames/X2_STFT_sliced.npy'

In [ ]:
freq_1 = stream_dominant_frequencies(sliced_combined[30000:45000], plot=False)
X_1 = create_dataset(freq_1)
np.save("X3_STFT_sliced.npy", X_1)  # Save

(15000, 3000, 1)


In [ ]:
del freq_1
del X_1

In [ ]:
freq_1 = stream_dominant_frequencies(sliced_combined[45000:60000], plot=False)
X_1 = create_dataset(freq_1)
np.save("X4_STFT_sliced.npy", X_1)  # Save

(15000, 3000, 1)


In [ ]:
del freq_1
del X_1

In [ ]:
# Move file to Google Drive
shutil.move("/content/X3_STFT_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames")
shutil.move("/content/X4_STFT_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames")

'/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames/X4_STFT_sliced.npy'

In [ ]:
freq_1 = stream_dominant_frequencies(sliced_combined[60000:75000], plot=False)
X_1 = create_dataset(freq_1)
np.save("X5_STFT_sliced.npy", X_1)  # Save

(15000, 3000, 1)


In [ ]:
del freq_1
del X_1

In [ ]:
freq_1 = stream_dominant_frequencies(sliced_combined[75000:], plot=False)
X_1 = create_dataset(freq_1)
np.save("X6_STFT_sliced.npy", X_1)  # Save

(348, 3000, 1)


In [ ]:
del freq_1
del X_1

In [ ]:
# Move file to Google Drive
shutil.move("/content/X5_STFT_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames")
shutil.move("/content/X6_STFT_sliced.npy", "/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames")

'/content/drive/MyDrive/data_codingan_skripsi/skripsi_stft_sliced_200frames/X6_STFT_sliced.npy'